In [58]:
import torch, pickle
import torch.nn as nn
import datetime

In [59]:
class Conv_BN_LeakyReLU(nn.Module):
    def __init__(self, in_channels, out_channels, ksize, padding=0, stride=1, dilation=1):
        super(Conv_BN_LeakyReLU, self).__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, ksize, padding=padding, stride=stride, dilation=dilation,
                     device='cuda:0', dtype=torch.float32),
            nn.BatchNorm2d(out_channels, device='cuda:0', dtype=torch.float32),
            nn.LeakyReLU(0.1, inplace=True)
        )

    def forward(self, x):
        return self.convs(x)

In [60]:
class DarkNet_19(nn.Module):
    def __init__(self):        
        super(DarkNet_19, self).__init__()
        # backbone network : DarkNet-19
        # output : stride = 2, c = 32
        self.conv_1 = nn.Sequential(
            Conv_BN_LeakyReLU(3, 32, 3, 1),
            nn.MaxPool2d((2,2), 2),
        )

        # output : stride = 4, c = 64
        self.conv_2 = nn.Sequential(
            Conv_BN_LeakyReLU(32, 64, 3, 1),
            nn.MaxPool2d((2,2), 2)
        )

        # output : stride = 8, c = 128
        self.conv_3 = nn.Sequential(
            Conv_BN_LeakyReLU(64, 128, 3, 1),
            Conv_BN_LeakyReLU(128, 64, 1),
            Conv_BN_LeakyReLU(64, 128, 3, 1),
            nn.MaxPool2d((2,2), 2)
        )

        # output : stride = 8, c = 256
        self.conv_4 = nn.Sequential(
            Conv_BN_LeakyReLU(128, 256, 3, 1),
            Conv_BN_LeakyReLU(256, 128, 1),
            Conv_BN_LeakyReLU(128, 256, 3, 1),
        )

        # output : stride = 16, c = 512
        self.maxpool_4 = nn.MaxPool2d((2, 2), 2)
        self.conv_5 = nn.Sequential(
            Conv_BN_LeakyReLU(256, 512, 3, 1),
            Conv_BN_LeakyReLU(512, 256, 1),
            Conv_BN_LeakyReLU(256, 512, 3, 1),
            Conv_BN_LeakyReLU(512, 256, 1),
            Conv_BN_LeakyReLU(256, 512, 3, 1),
        )
        
        # output : stride = 32, c = 1024
        self.maxpool_5 = nn.MaxPool2d((2, 2), 2)
        self.conv_6 = nn.Sequential(
            Conv_BN_LeakyReLU(512, 1024, 3, 1),
            Conv_BN_LeakyReLU(1024, 512, 1),
            Conv_BN_LeakyReLU(512, 1024, 3, 1),
            Conv_BN_LeakyReLU(1024, 512, 1),
            Conv_BN_LeakyReLU(512, 1024, 3, 1)
        )

        self.conv_7 = nn.Conv2d(1024, 1000, kernel_size=(1,1), stride=(1,1), padding=(0,0), bias=True,
                               device='cuda:0', dtype=torch.float32)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        c1 = self.conv_1(x)
        c2 = self.conv_2(c1)
        c3 = self.conv_3(c2)
        c3 = self.conv_4(c3)
        c4 = self.conv_5(self.maxpool_4(c3))
        c5 = self.conv_6(self.maxpool_5(c4))
        out = self.conv_7(c5)
        out = self.avg_pool(out)
        out = out.flatten(1)
        out = self.softmax(out)
        return out



In [47]:
def build_darknet19(pretrained=False):
    # model
    model = DarkNet_19()

    # load weight
    if pretrained:
        print('Loading pretrained weight ...')
        url = "https://github.com/yjh0410/image_classification_pytorch/releases/download/weight/darknet19.pth"
        # checkpoint state dict
        checkpoint_state_dict = torch.hub.load_state_dict_from_url(
            url=url, map_location="cpu", check_hash=True)
        # model state dict
        model_state_dict = model.state_dict()
        # check
        for k in list(checkpoint_state_dict.keys()):
            if k in model_state_dict:
                shape_model = tuple(model_state_dict[k].shape)
                shape_checkpoint = tuple(checkpoint_state_dict[k].shape)
                if shape_model != shape_checkpoint:
                    checkpoint_state_dict.pop(k)
            else:
                checkpoint_state_dict.pop(k)
                print(k)

        model.load_state_dict(checkpoint_state_dict)

    return model

In [48]:
import sys
sys.path.insert(1, '../../')

In [49]:
model = build_darknet19(pretrained=True)

Loading pretrained weight ...


In [50]:
import os
from data_preprocessing import *
data_path = '../../../datasets/imagenette2/'
norms_path = os.path.join(data_path, 'norms.json')
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.44969913363456726, 0.44868946075439453, 0.45163223147392273]
stds are: [0.28648287057876587, 0.28796446323394775, 0.2865694761276245]


In [51]:
from torchvision import transforms 
transformations_val = transforms.Compose([transforms.Resize((224, 224)),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize(mean=means, std=stds)
                                                 ])

In [52]:
val_dataset = ImageNetSubset(path=data_path, train=False, transform=transformations_val, half=False, show=False)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

In [53]:
next(model.parameters()).device

device(type='cuda', index=0)

In [57]:
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for i, (imgs, labels) in enumerate(val_loader):
        imgs = imgs.to('cuda:0')
        labels = labels.to('cuda:0')
        
        if (i+1) % 15 == 0:
            _datetime = datetime.datetime.now()
            print(f"{_datetime} Batch {i+1} ")

        outputs = model(imgs)
        preds = outputs.max(1)[1]

        total += labels.shape[0]
        correct += preds.eq(labels).sum().item()


accuracy = round(correct / total * 100., 4)
print(f"[Val] Accuracy: {accuracy}%")

2024-12-11 12:58:31.445458 Batch 15 
2024-12-11 12:58:36.836745 Batch 30 
2024-12-11 12:58:41.320199 Batch 45 
2024-12-11 12:58:44.542947 Batch 60 
[Val] Accuracy: 8.1529%


In [43]:
sum([p.numel() for p in Darknet19().parameters()])

19827616